In [30]:
import numpy as np
from gridworld import GridworldEnv


In [31]:
env = GridworldEnv()
print(env.nA)
print(env.shape)

4
[4, 4]


Policy is a matrix where each row specifies a state and each column is a possible action that can be taken in that state. The matrix provides the probability of taking an action when in a certain state.

Policy iteration has two steps to it -
a. Policy Evaluation - Iterative policy evaluation used to find the v(s)
b. Policy Improvement - Act greedily to find the action which gives the most expected reward and update the policy for that state to get an improved policy 

In [32]:
# Iterative policy evaluation 

def policy_eval(policy, env, discount_factor = 1.0, theta = 0.00001):
    V = np.zeros(env.nS)
    while True:
        delta = 0
        for s in range(env.nS):
            v = 0
            for a, action_prob in enumerate(policy[s]):
                #print(action, s, a)
                #for prob, next_state, reward, goal_or_not in env.P[s][a]:
                i = 0
                prob, next_state, reward, goal_or_not = env.P[s][a][i]
                i += 1
                v += action_prob * prob * (reward + discount_factor * V[next_state])
                    
            delta = max(delta, np.abs(v - V[s]))
            V[s] = v
        
        if delta < theta:
            break
    return np.array(V)

In [33]:
def action_value_calc(env, state, V, discount_factor):
    A = np.zeros(env.nA)
     
    for a in range(env.nA):
        i = 0
        prob, next_state, reward, done = env.P[state][a][i]
        i += 1
        A[a] += prob * (reward + discount_factor * V[next_state])
    return A   

def policy_improv(env, discount_factor = 1.0):
    policy = np.ones([env.nS, env.nA]) / env.nA
    
    while True:
        V = policy_eval(policy, env)
        policy_optimal = True
        
        for s in range(env.nS):
            chosen_action = np.argmax(policy[s])  # best action that can be chosen based on the current policy for that state
            
            A = action_value_calc(env, s, V, discount_factor) # 
            best_action = np.argmax(A) # best action value function index that can be chosen based on the state
            # np.argmax returns the index of the array
            
            if chosen_action != best_action:  # if both the actions are the same then the policy is the optimal policy else we need to converge further
                policy_optimal = False
            # np.eye creates an identity matrix with dimensions mxm as defined in env.nA - matrix with 1 on the diagonal
            # This line here below selects a row from an identity matrix based on the best action index that we get from the top
            policy[s] = np.eye(env.nA)[best_action] # updating the policy to get a better policy
        
        if policy_optimal == True:
            return policy, V

In [34]:
policy_improv(env)

(array([[1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.]]),
 array([ 0., -1., -2., -3., -1., -2., -3., -2., -2., -3., -2., -1., -3.,
        -2., -1.,  0.]))